In [6]:
import numpy as np, pandas as pd
from sklearn.datasets import load_breast_cancer, make_classification
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from scripts.classifiers import CostClassifierCV

In [2]:
def tune_threshold(model, X_train, y_train, X_test, y_test, alpha=2, n_thresholds=100):
    """Fit model, sweep thresholds, pick tau that maximizes cost-sensitive gain."""
    model = clone(model).fit(X_train, y_train)
    probs = model.predict_proba(X_test)[:, 1]
    taus = np.linspace(0, 1, n_thresholds)

    def gain(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        return -(fp + alpha * fn)

    best_gain, best_tau = -np.inf, 0.5
    for tau in taus:
        preds = (probs >= tau).astype(int)
        g = gain(y_test, preds)
        if g > best_gain:
            best_gain, best_tau = g, tau
    return best_tau, best_gain, probs

In [3]:
def run_experiment(X, y, dataset_name, alpha=3):
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import StandardScaler

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42)

    base_models = [
        make_pipeline(StandardScaler(), LogisticRegression(max_iter=500)),
        RandomForestClassifier(n_estimators=200, random_state=42),
        make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=5))
    ]

    results = []
    print(f"\n=== {dataset_name} ===")

    # Evaluate base models
    for model in base_models:
        name = model.steps[-1][0] if hasattr(model, "steps") else model.__class__.__name__
        results.extend(evaluate_model(name, model, X_train, y_train, X_test, y_test, alpha=alpha))

    # Evaluate ensembles
    results.extend(evaluate_costcv(base_models, X_train, y_train, X_test, y_test, alpha=alpha, method="dirichlet"))
    results.extend(evaluate_costcv(base_models, X_train, y_train, X_test, y_test, alpha=alpha, method="stacking"))

    df = pd.DataFrame(results)
    print("\nSummary Table:")
    display(df)
    return df

In [4]:
def evaluate_model(name, model, X_train, y_train, X_test, y_test, alpha=2, n_thresholds=100):
    """Evaluate a base model with untuned τ=0.5 and tuned τ maximizing gain."""
    model = clone(model).fit(X_train, y_train)
    probs = model.predict_proba(X_test)[:, 1]

    # Untuned (τ=0.5)
    preds_untuned = (probs >= 0.5).astype(int)

    # Tuned (sweep thresholds)
    tau_best, best_gain = 0.5, -np.inf
    taus = np.linspace(0, 1, n_thresholds)
    for tau in taus:
        preds = (probs >= tau).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
        g = -(fp + alpha * fn)
        if g > best_gain:
            best_gain, tau_best, preds_best = g, tau, preds

    def metrics(y_true, y_pred):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        acc = (tp + tn) / (tp + tn + fp + fn)
        prec = tp / (tp + fp + 1e-9)
        rec = tp / (tp + fn + 1e-9)
        f1 = 2 * prec * rec / (prec + rec + 1e-9)
        g = -(fp + alpha * fn)
        return acc, prec, rec, f1, g

    acc_u, prec_u, rec_u, f1_u, g_u = metrics(y_test, preds_untuned)
    acc_t, prec_t, rec_t, f1_t, g_t = metrics(y_test, preds_best)

    results = [
        {"Model": name, "Type": "Untuned", "Tau": 0.5, "Acc": acc_u, "Prec": prec_u, "Rec": rec_u, "F1": f1_u, "Gain": g_u},
        {"Model": name, "Type": "Tuned", "Tau": tau_best, "Acc": acc_t, "Prec": prec_t, "Rec": rec_t, "F1": f1_t, "Gain": g_t}
    ]
    return results


def evaluate_costcv(base_models, X_train, y_train, X_test, y_test, alpha=2, method="dirichlet"):
    clf = CostClassifierCV(base_models, alpha=alpha, random_state=42, method=method)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tp + tn) / (tp + tn + fp + fn)
    prec = tp / (tp + fp + 1e-9)
    rec = tp / (tp + fn + 1e-9)
    f1 = 2 * prec * rec / (prec + rec + 1e-9)
    g = -(fp + alpha * fn)

    results = [{
        "Model": f"Ensemble-{method}", "Type": "Tuned", "Tau": clf.threshold_,
        "Acc": acc, "Prec": prec, "Rec": rec, "F1": f1, "Gain": g
    }]
    return results

In [7]:
# Breast Cancer dataset
X, y = load_breast_cancer(return_X_y=True)
df_bc = run_experiment(X, y, "Breast Cancer", alpha=3)


=== Breast Cancer ===

Summary Table:


,Model,Type,Tau,Acc,Prec,Rec,F1,Gain
0,logisticregression,Untuned,0.500000,0.988304,0.990654,0.990654,0.990654,-4
1,logisticregression,Tuned,0.363636,0.994152,0.990741,1.000000,0.995349,-1
2,RandomForestClassifier,Untuned,0.500000,0.941520,0.944954,0.962617,0.953704,-18
3,RandomForestClassifier,Tuned,0.272727,0.953216,0.938053,0.990654,0.963636,-10
4,kneighborsclassifier,Untuned,0.500000,0.959064,0.938596,1.000000,0.968326,-7
5,kneighborsclassifier,Tuned,0.404040,0.959064,0.938596,1.000000,0.968326,-7
6,Ensemble-dirichlet,Tuned,0.525253,0.994152,0.990741,1.000000,0.995349,-1
7,Ensemble-stacking,Tuned,0.515152,0.994152,0.990741,1.000000,0.995349,-1


In [8]:
# Synthetic imbalanced dataset
X2, y2 = make_classification(n_samples=2000, n_features=20, n_informative=10,
                             n_redundant=5, n_classes=2, weights=[0.9, 0.1],
                             flip_y=0.01, random_state=42)
df_syn = run_experiment(X2, y2, "Synthetic Imbalanced", alpha=5)


=== Synthetic Imbalanced ===

Summary Table:


,Model,Type,Tau,Acc,Prec,Rec,F1,Gain
0,logisticregression,Untuned,0.500000,0.926667,0.750000,0.435484,0.551020,-184
1,logisticregression,Tuned,0.202020,0.896667,0.500000,0.758065,0.602564,-122
2,RandomForestClassifier,Untuned,0.500000,0.926667,0.846154,0.354839,0.500000,-204
3,RandomForestClassifier,Tuned,0.151515,0.866667,0.432836,0.935484,0.591837,-96
4,kneighborsclassifier,Untuned,0.500000,0.936667,0.928571,0.419355,0.577778,-182
5,kneighborsclassifier,Tuned,0.010101,0.871667,0.438017,0.854839,0.579235,-113
6,Ensemble-dirichlet,Tuned,0.141414,0.911667,0.547368,0.838710,0.662420,-93
7,Ensemble-stacking,Tuned,0.121212,0.893333,0.491228,0.903226,0.636364,-88
